# Amazon_Product_Recommendation_System

So this is a very interesting problem of Recommendation System, Recommendation System is now a days very very common whether its e-commerce companies like Amazon, Flipkart.  

# Business Objective

The main objective of our recommendation system is to enhance the customer shopping experience and increase sales by implementing a personalized product recommendation system that leverages advanced algorithms and data analytics. The system should more helpful and engaging by recommending products that our customers are most likely to be interested in. By doing this, we aim to boost sales and create a better shopping experience.

# Constaints

* Data Quality and availability
* Scalability
* Should integrate smoothly
* Accuracy and diversity
* Algorithm Complexity
* A/B Testing and Continuous Improvement

# Machine Learning Formulation

* Problem Defination
* Data Collection and Preprocessing 
* Data Cleaning 
* Analysing and Filtering the data
* building and testing the model
* deploy and integrate the model 

# Data

In [1]:
#importing the libraries

from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import math
import time
import re
import os
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances
from matplotlib import gridspec
from scipy.sparse import hstack
warnings.filterwarnings("ignore")

In [2]:
# loading the data
data = pd.read_json('tops_fashion.json')



In [3]:
# Data overview
data.head() 

,sku,asin,product_type_name,formatted_price,author,color,brand,publisher,availability,reviews,large_image_url,availability_type,small_image_url,editorial_review,title,model,medium_image_url,manufacturer,editorial_reivew
0,None,B016I2TS4W,SHIRT,None,None,None,FNC7C,None,None,"[False, https://www.amazon.com/reviews/iframe?...",https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,Minions Como Superheroes Ironman Women's O Nec...,Minions Como Superheroes Ironman Long Sleeve R...,None,https://images-na.ssl-images-amazon.com/images...,None,NaN
1,None,B01N49AI08,SHIRT,None,None,None,FIG Clothing,None,None,"[False, https://www.amazon.com/reviews/iframe?...",https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,Sizing runs on the small side. FIG® recommends...,FIG Clothing Womens Izo Tunic,None,https://images-na.ssl-images-amazon.com/images...,None,NaN
2,None,B01JDPCOHO,SHIRT,None,None,None,FIG Clothing,None,None,"[False, https://www.amazon.com/reviews/iframe?...",https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,Sizing runs on the small side. FIG® recommends...,FIG Clothing Womens Won Top,None,https://images-na.ssl-images-amazon.com/images...,None,NaN
3,None,B01N19U5H5,SHIRT,None,None,None,Focal18,None,None,"[True, https://www.amazon.com/reviews/iframe?a...",https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,100% Brand New & Fashion<br> Quantity: 1 Piece...,Focal18 Sailor Collar Bubble Sleeve Blouse Shi...,None,https://images-na.ssl-images-amazon.com/images...,None,NaN
4,None,B004GSI2OS,SHIRT,$26.26,None,Onyx Black/ Stone,FeatherLite,None,Usually ships in 6-10 business days,"[False, https://www.amazon.com/reviews/iframe?...",https://images-na.ssl-images-amazon.com/images...,now,https://images-na.ssl-images-amazon.com/images...,,Featherlite Ladies' Long Sleeve Stain Resistan...,None,https://images-na.ssl-images-amazon.com/images...,None,NaN


In [4]:
data.columns

Index(['sku', 'asin', 'product_type_name', 'formatted_price', 'author',
       'color', 'brand', 'publisher', 'availability', 'reviews',
       'large_image_url', 'availability_type', 'small_image_url',
       'editorial_review', 'title', 'model', 'medium_image_url',
       'manufacturer', 'editorial_reivew'],
      dtype='object')

In [5]:
data.shape

(183138, 19)

In [6]:
print ('Number of data points : ', data.shape[0], \
       'Number of features/variables:', data.shape[1])

Number of data points :  183138 Number of features/variables: 19


In [7]:
# We'll be using only below 7 variables from 19 variable
data = data[['asin', 'brand', 'color', 'medium_image_url', 'product_type_name', 'title', 'formatted_price']]

In [8]:
print ('Number of data points : ', data.shape[0], \
       'Number of features:', data.shape[1])
data.head() 

Number of data points :  183138 Number of features: 7


,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
0,B016I2TS4W,FNC7C,None,https://images-na.ssl-images-amazon.com/images...,SHIRT,Minions Como Superheroes Ironman Long Sleeve R...,None
1,B01N49AI08,FIG Clothing,None,https://images-na.ssl-images-amazon.com/images...,SHIRT,FIG Clothing Womens Izo Tunic,None
2,B01JDPCOHO,FIG Clothing,None,https://images-na.ssl-images-amazon.com/images...,SHIRT,FIG Clothing Womens Won Top,None
3,B01N19U5H5,Focal18,None,https://images-na.ssl-images-amazon.com/images...,SHIRT,Focal18 Sailor Collar Bubble Sleeve Blouse Shi...,None
4,B004GSI2OS,FeatherLite,Onyx Black/ Stone,https://images-na.ssl-images-amazon.com/images...,SHIRT,Featherlite Ladies' Long Sleeve Stain Resistan...,$26.26


Lets jump to each feature and understand data better

Basic stats for feature product_type_name

In [9]:
print(data['product_type_name'].describe())

# 91.62% (167794/183138) of the products are shirts,



count     183138
unique        72
top        SHIRT
freq      167794
Name: product_type_name, dtype: object


In [10]:
print(data['product_type_name'].unique())

['SHIRT' 'SWEATER' 'APPAREL' 'OUTDOOR_RECREATION_PRODUCT'
 'BOOKS_1973_AND_LATER' 'PANTS' 'HAT' 'SPORTING_GOODS' 'DRESS' 'UNDERWEAR'
 'SKIRT' 'OUTERWEAR' 'BRA' 'ACCESSORY' 'ART_SUPPLIES' 'SLEEPWEAR'
 'ORCA_SHIRT' 'HANDBAG' 'PET_SUPPLIES' 'SHOES' 'KITCHEN' 'ADULT_COSTUME'
 'HOME_BED_AND_BATH' 'MISC_OTHER' 'BLAZER' 'HEALTH_PERSONAL_CARE'
 'TOYS_AND_GAMES' 'SWIMWEAR' 'CONSUMER_ELECTRONICS' 'SHORTS' 'HOME'
 'AUTO_PART' 'OFFICE_PRODUCTS' 'ETHNIC_WEAR' 'BEAUTY'
 'INSTRUMENT_PARTS_AND_ACCESSORIES' 'POWERSPORTS_PROTECTIVE_GEAR' 'SHIRTS'
 'ABIS_APPAREL' 'AUTO_ACCESSORY' 'NONAPPARELMISC' 'TOOLS' 'BABY_PRODUCT'
 'SOCKSHOSIERY' 'POWERSPORTS_RIDING_SHIRT' 'EYEWEAR' 'SUIT'
 'OUTDOOR_LIVING' 'POWERSPORTS_RIDING_JACKET' 'HARDWARE' 'SAFETY_SUPPLY'
 'ABIS_DVD' 'VIDEO_DVD' 'GOLF_CLUB' 'MUSIC_POPULAR_VINYL'
 'HOME_FURNITURE_AND_DECOR' 'TABLET_COMPUTER' 'GUILD_ACCESSORIES'
 'ABIS_SPORTS' 'ART_AND_CRAFT_SUPPLY' 'BAG' 'MECHANICAL_COMPONENTS'
 'SOUND_AND_RECORDING_EQUIPMENT' 'COMPUTER_COMPONENT' 'JEWELRY'
 'B

In [11]:
product_list=Counter(data['product_type_name'])
print(product_list.most_common(10))

# this will give product_wise Count of each category

[('SHIRT', 167794), ('APPAREL', 3549), ('BOOKS_1973_AND_LATER', 3336), ('DRESS', 1584), ('SPORTING_GOODS', 1281), ('SWEATER', 837), ('OUTERWEAR', 796), ('OUTDOOR_RECREATION_PRODUCT', 729), ('ACCESSORY', 636), ('UNDERWEAR', 425)]


Basic stats for feature brand


In [12]:
print(data['brand'].describe())

count     182987
unique     10577
top         Zago
freq         223
Name: brand, dtype: object


In [13]:
brand_count = Counter(list(data['brand']))
brand_count.most_common(10)

#This will give count of each brand

[('Zago', 223),
 ('XQS', 222),
 ('Yayun', 215),
 ('YUNY', 198),
 ('XiaoTianXin-women clothes', 193),
 ('Generic', 192),
 ('Boohoo', 190),
 ('Alion', 188),
 ('Abetteric', 187),
 ('TheMogan', 187)]

Basic feature for feature color

In [14]:
print(data['color'].describe())


# we have 7380 unique colors
# 7.2% of products are black in color
# 64956 of 183138 products have brand information. That's approx 35.4%.

count     64956
unique     7380
top       Black
freq      13207
Name: color, dtype: object


In [15]:
color_count = Counter(list(data['color']))
color_count.most_common(10)

# This will give the colorwise Count

[(None, 118182),
 ('Black', 13207),
 ('White', 8616),
 ('Blue', 3570),
 ('Red', 2289),
 ('Pink', 1842),
 ('Grey', 1499),
 ('*', 1388),
 ('Green', 1258),
 ('Multi', 1203)]

Basic stats for feature:formatted price


In [16]:
 
print(data['formatted_price'].describe())

# Only 28,395 (15.5% of whole data) products with price information

count      28395
unique      3135
top       $19.99
freq         945
Name: formatted_price, dtype: object


In [17]:
price_count = Counter(list(data['formatted_price']))
price_count.most_common(10)

# this will give count of each price varient.

[(None, 154743),
 ('$19.99', 945),
 ('$9.99', 749),
 ('$9.50', 601),
 ('$14.99', 472),
 ('$7.50', 463),
 ('$24.99', 414),
 ('$29.99', 370),
 ('$8.99', 343),
 ('$9.01', 336)]

Basic stats for feature: Title

In [18]:
print(data['title'].describe())

# All of the products have a title. 
# Titles are fairly descriptive of what the product is. 
# We use titles extensively in this workshop 
# as they are short and informative.



count                                                183138
unique                                               175985
top       Nakoda Cotton Self Print Straight Kurti For Women
freq                                                     77
Name: title, dtype: object


In [19]:
# Duplicate items
data['title'].duplicated().sum()

7153

In [20]:
# Duplicate items with different colors so total we have 2734 items have same title but different color
same_title_diff_color = data.groupby('title').filter(lambda x: x['color'].nunique() > 1)
same_title_diff_color.shape

(2734, 7)

In [21]:
# consider products which have price information
# data['formatted_price'].isnull() => gives the information 
data = data.loc[~data['formatted_price'].isnull()]
print('Number of data points After eliminating price=NULL :', data.shape[0])

Number of data points After eliminating price=NULL : 28395


In [22]:
# consider products which have color information
data =data.loc[~data['color'].isnull()]
print('Number of data points After eliminating color=NULL :', data.shape[0])

Number of data points After eliminating color=NULL : 28385


We brought down the number of data points from 183K to 28K.


# DATA CLEANING

In [23]:
#removing all the title with less than 4 words

data=data[data['title'].apply(lambda x: len(x.split())>4)]

In [24]:
data.shape #now we have 27949 rows

(27949, 7)

In [25]:
#now let's sort the data based on title(alphabetical order)
data.sort_values("title", inplace=True,ascending=False)

In [26]:
data.shape

(27949, 7)

In [27]:
data.head()

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
61973,B06Y1KZ2WB,Éclair,Black/Pink,https://images-na.ssl-images-amazon.com/images...,SHIRT,Éclair Women's Printed Thin Strap Blouse Black...,$24.99
133820,B010RV33VE,xiaoming,Pink,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Womens Sleeveless Loose Long T-shirts...,$18.19
81461,B01DDSDLNS,xiaoming,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Women's White Long Sleeve Single Brea...,$21.58
75995,B00X5LYO9Y,xiaoming,Red Anchors,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Stripes Tank Patch/Bear Sleeve Anchor...,$15.91
151570,B00WPJG35K,xiaoming,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Sleeve Sheer Loose Tassel Kimono Woma...,$14.32


In [28]:
data.title[21]

'Supernatural Chibis Sam Dean And Castiel Short Sleeve T Shirts For Lady Purple S'

In [29]:
data['Gender'] = data['title'].apply(lambda x: 'female' if ('ladies' in x.lower() or 'women' in x.lower() or 'top' in x.lower())else 'male')

In [30]:
data.head()

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price,Gender
61973,B06Y1KZ2WB,Éclair,Black/Pink,https://images-na.ssl-images-amazon.com/images...,SHIRT,Éclair Women's Printed Thin Strap Blouse Black...,$24.99,female
133820,B010RV33VE,xiaoming,Pink,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Womens Sleeveless Loose Long T-shirts...,$18.19,female
81461,B01DDSDLNS,xiaoming,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Women's White Long Sleeve Single Brea...,$21.58,female
75995,B00X5LYO9Y,xiaoming,Red Anchors,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Stripes Tank Patch/Bear Sleeve Anchor...,$15.91,female
151570,B00WPJG35K,xiaoming,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Sleeve Sheer Loose Tassel Kimono Woma...,$14.32,male


In [31]:
def categories_sleeve(title):
    title = title.lower()
    if 'short sleeve' in title:
        return 'short sleeve'
    
    if 'long sleeve' in title:
        return 'long sleeve'
    
    if 'sleeveless'and 'tank' in title:
        return 'sleeveless'
    

In [32]:
data['sleeve_type'] = data['title'].apply(categories_sleeve)

In [33]:
data.head()

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price,Gender,sleeve_type
61973,B06Y1KZ2WB,Éclair,Black/Pink,https://images-na.ssl-images-amazon.com/images...,SHIRT,Éclair Women's Printed Thin Strap Blouse Black...,$24.99,female,None
133820,B010RV33VE,xiaoming,Pink,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Womens Sleeveless Loose Long T-shirts...,$18.19,female,None
81461,B01DDSDLNS,xiaoming,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Women's White Long Sleeve Single Brea...,$21.58,female,long sleeve
75995,B00X5LYO9Y,xiaoming,Red Anchors,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Stripes Tank Patch/Bear Sleeve Anchor...,$15.91,female,sleeveless
151570,B00WPJG35K,xiaoming,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Sleeve Sheer Loose Tassel Kimono Woma...,$14.32,male,None


In [34]:
data['sleeve_type'].value_counts()

sleeve_type
sleeveless      3812
long sleeve     2464
short sleeve    1683
Name: count, dtype: int64

In [35]:
data['sleeve_type'].isnull().sum()

19990

In [36]:
product_type_count = data['product_type_name'].value_counts()

In [37]:
data_filtered = data[data['product_type_name'].isin(product_type_count[product_type_count > 10].index)]


In [38]:
data_filtered['product_type_name'].value_counts()

product_type_name
SHIRT                          21162
BOOKS_1973_AND_LATER            3327
APPAREL                         1038
ACCESSORY                        553
SPORTING_GOODS                   362
OUTERWEAR                        334
DRESS                            173
SWEATER                          147
BLAZER                           127
OUTDOOR_RECREATION_PRODUCT       104
HEALTH_PERSONAL_CARE              79
TOYS_AND_GAMES                    77
MISC_OTHER                        62
UNDERWEAR                         57
SLEEPWEAR                         49
AUTO_ACCESSORY                    43
PANTS                             40
ETHNIC_WEAR                       34
POWERSPORTS_PROTECTIVE_GEAR       30
SKIRT                             24
BRA                               15
HANDBAG                           14
TOOLS                             13
Name: count, dtype: int64

In [39]:
data= data_filtered

In [40]:
data['product_type_name'].value_counts()

product_type_name
SHIRT                          21162
BOOKS_1973_AND_LATER            3327
APPAREL                         1038
ACCESSORY                        553
SPORTING_GOODS                   362
OUTERWEAR                        334
DRESS                            173
SWEATER                          147
BLAZER                           127
OUTDOOR_RECREATION_PRODUCT       104
HEALTH_PERSONAL_CARE              79
TOYS_AND_GAMES                    77
MISC_OTHER                        62
UNDERWEAR                         57
SLEEPWEAR                         49
AUTO_ACCESSORY                    43
PANTS                             40
ETHNIC_WEAR                       34
POWERSPORTS_PROTECTIVE_GEAR       30
SKIRT                             24
BRA                               15
HANDBAG                           14
TOOLS                             13
Name: count, dtype: int64

In [41]:
data.head()

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price,Gender,sleeve_type
61973,B06Y1KZ2WB,Éclair,Black/Pink,https://images-na.ssl-images-amazon.com/images...,SHIRT,Éclair Women's Printed Thin Strap Blouse Black...,$24.99,female,None
133820,B010RV33VE,xiaoming,Pink,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Womens Sleeveless Loose Long T-shirts...,$18.19,female,None
81461,B01DDSDLNS,xiaoming,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Women's White Long Sleeve Single Brea...,$21.58,female,long sleeve
75995,B00X5LYO9Y,xiaoming,Red Anchors,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Stripes Tank Patch/Bear Sleeve Anchor...,$15.91,female,sleeveless
151570,B00WPJG35K,xiaoming,White,https://images-na.ssl-images-amazon.com/images...,SHIRT,xiaoming Sleeve Sheer Loose Tassel Kimono Woma...,$14.32,male,None


In [42]:
data[['title', 'Gender']].sample(20)

,title,Gender
173413,Stylish Shirt Collar Necklace Detachable Neckb...,male
61151,North End Ladies Performance Embossed Polo Shi...,female
24712,DZT1968® Punk Goth Lady Disco Dance Lace Finge...,male
60828,Exotic India Golden Designer Choli with Embroi...,male
20780,Niceda Women's Captain Morgan Art Tank Top T S...,female
160143,Women's Sorry I'm Late I Didn't Want To Come C...,female
152877,Basic Solid Sleeveless Crop Tank Tops Black Si...,female
71406,Victoria's Secret Pink Tee Shirt M Pink For Women,female
7097,"Alice + Olivia Womens Ian Silk Top, Xs, White",female
140944,Hippie Rose Juniors Buttons Lace Top Blouse Cl...,female


In [43]:
data['Gender'].value_counts()

Gender
female    22448
male       5416
Name: count, dtype: int64

In [44]:
import warnings
warnings.filterwarnings("ignore")
import nltk
nltk.download("stopwords")


[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [45]:
data.columns

Index(['asin', 'brand', 'color', 'medium_image_url', 'product_type_name',
       'title', 'formatted_price', 'Gender', 'sleeve_type'],
      dtype='object')

In [46]:
# we use the list of stop words that are downloaded from nltk lib.
stop_words = set(stopwords.words('english'))

# Preprocessing function
def nlp_preprocessing(total_text):
    if isinstance(total_text, str):
        processed_text = ""
        for word in total_text.split():
            word = "".join(e for e in word if e.isalnum())  # Remove special chars
            word = word.lower()  # Convert to lowercase
            if word not in stop_words:  # Stopword removal
                processed_text += word + " "
        return processed_text.strip()
    else:
        return total_text

columns_to_process = ['asin', 'brand', 'color', 'medium_image_url', 'product_type_name',
       'title', 'formatted_price']

for column in columns_to_process:
    data[column] = data[column].apply(nlp_preprocessing)
    
print(data)

              asin               brand                 color   
61973   b06y1kz2wb              éclair             blackpink  \
133820  b010rv33ve            xiaoming                  pink   
81461   b01ddsdlns            xiaoming                 white   
75995   b00x5lyo9y            xiaoming           red anchors   
151570  b00wpjg35k            xiaoming                 white   
...            ...                 ...                   ...   
10679   b06wp94nlg                  aj                 black   
12050   b06wrw8rq1                  aj                 black   
40451   b073sknqhd    workout princess  premium heather gray   
109599  b00ki3vdxm         crazy4bling                purple   
118987  b008d30agk  outofprintclothing          multicolored   

                                         medium_image_url product_type_name   
61973   httpsimagesnasslimagesamazoncomimagesi41qzyliv...             shirt  \
133820  httpsimagesnasslimagesamazoncomimagesi41m2reux...             shi

In [47]:
data.head()

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price,Gender,sleeve_type
61973,b06y1kz2wb,éclair,blackpink,httpsimagesnasslimagesamazoncomimagesi41qzyliv...,shirt,éclair womens printed thin strap blouse black ...,2499,female,None
133820,b010rv33ve,xiaoming,pink,httpsimagesnasslimagesamazoncomimagesi41m2reux...,shirt,xiaoming womens sleeveless loose long tshirts ...,1819,female,None
81461,b01ddsdlns,xiaoming,white,httpsimagesnasslimagesamazoncomimagesi51c4p2b6...,shirt,xiaoming womens white long sleeve single breas...,2158,female,long sleeve
75995,b00x5lyo9y,xiaoming,red anchors,httpsimagesnasslimagesamazoncomimagesi51rlutsk...,shirt,xiaoming stripes tank patchbear sleeve anchor ...,1591,female,sleeveless
151570,b00wpjg35k,xiaoming,white,httpsimagesnasslimagesamazoncomimagesi41qkcbhq...,shirt,xiaoming sleeve sheer loose tassel kimono woma...,1432,male,None


In [48]:
from nltk.stem.porter import *
stemmer = PorterStemmer()
print(stemmer.stem('arguing'))
print(stemmer.stem('fishing'))


# We tried using stemming on our titles and it didnot work very well. 



argu
fish


In [49]:
import re
from collections import Counter
import math

# Function to convert text into a vector of word counts
def text_to_vector(text):
    word = re.compile(r'\w+')
    words = word.findall(text)  # Find all words in the text
    return Counter(words)  # Count the occurrence of each word

# Function to compute cosine similarity between two vectors
def cosine_similarity(vector1, vector2):
    # Find the common words between both vectors
    common_words = set(vector1.keys()) & set(vector2.keys())
    
    # Calculate the dot product between the two vectors
    dot_product = sum([vector1[word] * vector2[word] for word in common_words])

    # Calculate the magnitudes (lengths) of the vectors
    magnitude1 = math.sqrt(sum([count**2 for count in vector1.values()]))
    magnitude2 = math.sqrt(sum([count**2 for count in vector2.values()]))

    # Handle the case when one of the vectors has zero magnitude
    if magnitude1 == 0 or magnitude2 == 0:
        return 0.0

    # Return cosine similarity score
    return dot_product / (magnitude1 * magnitude2)

# Function to get the result of text similarity between two documents
def get_result(doc_id, content_a, content_b, url, model):
    # Convert texts to vectors
    vector1 = text_to_vector(content_a)
    vector2 = text_to_vector(content_b)
    
    # Compute cosine similarity between the two text vectors
    similarity_score = cosine_similarity(vector1, vector2)
    
    # Return or print the result (you can modify the return statement based on your needs)
    result = {
        "doc_id": doc_id,
        "url": url,
        "similarity_score": similarity_score,
        "model": model  # Placeholder to include the model if needed
    }
    return result



In [50]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd


# Initialize CountVectorizer
title_vectorizer = CountVectorizer()

# Apply CountVectorizer on the 'title' column to get term-frequency matrix
title_features = title_vectorizer.fit_transform(data['title'])

# Check the shape of the matrix
print(title_features.get_shape())

(27864, 13695)


In [51]:
title_features[932]

<1x13695 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [52]:
data.iloc[932]['asin']

'b06y6g64r4'

In [53]:
import numpy as np
from sklearn.metrics import pairwise_distances
import pandas as pd

# Assuming title_features contains the title vectors generated using CountVectorizer

def bag_of_words_model(doc_id, num_results):
    # Calculate pairwise cosine distances from the given apparel to all other apparels
    pairwise_dist = pairwise_distances(title_features, title_features[doc_id])
    
    # Sort the distances to get indices of the smallest distances (most similar items)
    indices = np.argsort(pairwise_dist.flatten())[0:num_results]
    
    # Sort the distances to get the smallest distances
    pdists = np.sort(pairwise_dist.flatten())[0:num_results]
    
    # Dataframe indices corresponding to the smallest distances
    df_indices = list(data.index[indices])
    
    # Print results for the similar products
    for i in range(0, len(indices)):
        # Fetch the result and print product information
        get_result(
            indices[i],
            data['title'].loc[df_indices[0]],       # Original apparel's title
            data['title'].loc[df_indices[i]],       # Similar apparel's title
            data['medium_image_url'].loc[df_indices[i]],  # Image URL of the similar product
            'bag_of_words'  # Model name
        )
        print('ASIN :', data['asin'].loc[df_indices[i]])
        print('Brand:', data['brand'].loc[df_indices[i]])
        print('Title:', data['title'].loc[df_indices[i]])
        print('Cosine similarity with the query point:', pdists[i])
        print('='*60)

# Example usage: calling the bag-of-words model for a product with ID 932 to get 10 similar products
bag_of_words_model(932, 10)

ASIN : b06y6g64r4
Brand: xhilaration
Title: xhilaration womens floral blouse ivory spring medium
Cosine similarity with the query point: 0.0
ASIN : b06zz2frsb
Brand: xhilaration
Title: xhilaration womens blouse black medium
Cosine similarity with the query point: 2.0
ASIN : b071cthvtv
Brand: xhilaration
Title: xhilaration womens floral pleated blouse xsmall ivory floral
Cosine similarity with the query point: 2.23606797749979
ASIN : b01nar8ce0
Brand: sactuary
Title: sanctuary womens scarlet blouse medium
Cosine similarity with the query point: 2.449489742783178
ASIN : b071xt7pgn
Brand: xhilaration
Title: xhilaration womens blouse black xxl
Cosine similarity with the query point: 2.449489742783178
ASIN : b071zvr3tv
Brand: xhilaration
Title: xhilaration womens sequin sleeveless blouse goldblack medium
Cosine similarity with the query point: 2.449489742783178
ASIN : b06y674b25
Brand: xhilaration
Title: xhilaration womens bell sleeve blouse xsmall ivory
Cosine similarity with the query poi

In [54]:
bag_of_words_model(12566, 20)

ASIN : b0755pxb3g
Brand: melissa mccarthy seven7
Title: melissa mccarthy seven7 trendy womens plus size ribbed high low pink shirt top 2x
Cosine similarity with the query point: 0.0
ASIN : b0755lgtd9
Brand: melissa mccarthy seven7
Title: melissa mccarthy seven7 trendy woemsn plus size high low shirt top 0x
Cosine similarity with the query point: 2.449489742783178
ASIN : b06xv1g5yr
Brand: melissa mccarthy seven7
Title: melissa mccarthy seven7 plus size pleated high low top size 1x
Cosine similarity with the query point: 3.0
ASIN : b06xtrlgp8
Brand: melissa mccarthy seven7
Title: melissa mccarthy seven7 womens plus pleated front crepe tunic top pink 2x
Cosine similarity with the query point: 3.1622776601683795
ASIN : b0759yypr1
Brand: melissa mccarthy seven7
Title: melissa mccarthy seven7 trendy plus size scarf neck blouse 2x rose
Cosine similarity with the query point: 3.3166247903554
ASIN : b01n7atp2x
Brand: melissa mccarthy seven7
Title: melissa mccarthy seven7 womens plus size vneck 

In [55]:
tfidf_title_vectorizer = TfidfVectorizer(min_df = 0.0)
tfidf_title_features = tfidf_title_vectorizer.fit_transform(data['title'])
# tfidf_title_features.shape = #data_points * #words_in_corpus
# CountVectorizer().fit_transform(courpus) returns the a sparase matrix of dimensions #data_points * #words_in_corpus
# tfidf_title_features[doc_id, index_of_word_in_corpus] = tfidf values of the word in given doc

In [56]:
from sklearn.metrics import pairwise_distances
import numpy as np

def tfidf_model(doc_id, num_results):
    # Calculate pairwise cosine distances from the given apparel to all other apparels
    pairwise_dist = pairwise_distances(tfidf_title_features, tfidf_title_features[doc_id])
    
    # Sort the distances to get indices of the smallest distances (most similar items)
    indices = np.argsort(pairwise_dist.flatten())[0:num_results]
    
    # Sort the distances to get the smallest distances
    pdists = np.sort(pairwise_dist.flatten())[0:num_results]
    
    # Dataframe indices corresponding to the smallest distances
    df_indices = list(data.index[indices])

    # Print results for the similar products
    for i in range(0, len(indices)):
        # Fetch the result and print product information
        get_result(
            indices[i],
            data['title'].loc[df_indices[0]],  # Original apparel's title
            data['title'].loc[df_indices[i]],  # Similar apparel's title
            data['medium_image_url'].loc[df_indices[i]],  # Image URL of the similar product
            'tfidf'  # Model name
        )
        print('ASIN :', data['asin'].loc[df_indices[i]])
        print('BRAND :', data['brand'].loc[df_indices[i]])
        print('Cosine similarity with the query point:', pdists[i])
        print('='*125)

# Example usage: calling the tfidf_model for a product with ID 932 to get 10 similar products
tfidf_model(932, 10)


ASIN : b06y6g64r4
BRAND : xhilaration
Cosine similarity with the query point: 0.0
ASIN : b071cthvtv
BRAND : xhilaration
Cosine similarity with the query point: 0.816494216115052
ASIN : b06zz2frsb
BRAND : xhilaration
Cosine similarity with the query point: 0.857673303321785
ASIN : b06y674b25
BRAND : xhilaration
Cosine similarity with the query point: 0.943429496806633
ASIN : b072bnvldw
BRAND : xhilaration
Cosine similarity with the query point: 1.0095188784168831
ASIN : b071xt7pgn
BRAND : xhilaration
Cosine similarity with the query point: 1.010086224243027
ASIN : b06y1w9sss
BRAND : xhilaration
Cosine similarity with the query point: 1.0649674826245168
ASIN : b071j26678
BRAND : dantelle
Cosine similarity with the query point: 1.0808640456788396
ASIN : b072m9twsk
BRAND : xhilaration
Cosine similarity with the query point: 1.0835805128608553
ASIN : b06xbs17rz
BRAND : xhilaration
Cosine similarity with the query point: 1.0841027780584926


In [57]:
import pickle
with open('word2vec_model', 'rb') as handle:
    model = pickle.load(handle)

In [58]:
def get_word_vec(sentence, doc_id, m_name):
    # m_name: model information it will take two values
        # if  m_name == 'avg', we will append the model[i], w2v representation of word i
        # if m_name == 'weighted', we will multiply each w2v[word] with the idf(word)
    vec = []
    for i in sentence.split():
        if i in vocab:
            if m_name == 'weighted' and i in  idf_title_vectorizer.vocabulary_:
                vec.append(idf_title_features[doc_id, idf_title_vectorizer.vocabulary_[i]] * model[i])
            elif m_name == 'avg':
                vec.append(model[i])
        else:
            # if the word in our courpus is not there in the google word2vec corpus, we are just ignoring it
            vec.append(np.zeros(shape=(300,)))
    # we will return a numpy array of shape (#number of words in title * 300 ) 300 = len(w2v_model[word])
    # each row represents the word2vec representation of each word (weighted/avg) in given sentance 
    return  np.array(vec)

def get_distance(vec1, vec2):
    # vec1 = np.array(#number_of_words_title1 * 300), each row is a vector of length 300 corresponds to each word in give title
    # vec2 = np.array(#number_of_words_title2 * 300), each row is a vector of length 300 corresponds to each word in give title
    
    final_dist = []
    # for each vector in vec1 we caluclate the distance(euclidean) to all vectors in vec2
    for i in vec1:
        dist = []
        for j in vec2:
            # np.linalg.norm(i-j) will result the euclidean distance between vectors i, j
            dist.append(np.linalg.norm(i-j))
        final_dist.append(np.array(dist))
    # final_dist = np.array(#number of words in title1 * #number of words in title2)
    # final_dist[i,j] = euclidean distance between vectors i, j
    return np.array(final_dist)




In [59]:
# vocab = stores all the words that are there in google w2v model
# vocab = model.wv.vocab.keys() # if you are using Google word2Vec
vocab = model.keys() 

def build_avg_vec(sentence, num_features, doc_id, m_name):
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0

    for word in sentence.split():
        if word in vocab:
            if m_name == 'weighted' and word in idf_title_vectorizer.vocabulary_:
                featureVec = np.add(featureVec, idf_title_features[doc_id, idf_title_vectorizer.vocabulary_[word]] * model[word])
            elif m_name == 'avg':
                featureVec = np.add(featureVec, model[word])
            nwords += 1
    
    if nwords > 0:
        featureVec = np.divide(featureVec, nwords)
    
    return featureVec


In [60]:
doc_id = 0
w2v_title = []

# Build average vector representation for each title
for title in data['title']:
    avg_vec = build_avg_vec(title, 300, doc_id, 'avg')
    w2v_title.append(avg_vec)
    doc_id += 1

# Converting list of vectors to a NumPy array
w2v_title = np.array(w2v_title)

In [61]:
def avg_w2v_model(doc_id, num_results):
    # Ensure num_results is within bounds
    if num_results > len(w2v_title):
        raise ValueError("num_results exceeds the number of available documents")
    
    
    pairwise_dist = pairwise_distances(w2v_title, w2v_title[doc_id].reshape(1, -1)) # Pairwise Distance
    
    # indices of the smallest distances
    indices = np.argsort(pairwise_dist.flatten())[0:num_results]
    pdists = np.sort(pairwise_dist.flatten())[0:num_results]
    
    # DataFrame indices
    df_indices = list(data.index[indices])
    
    #Results
    for i in range(len(indices)):
        print('ASIN :', data['asin'].loc[df_indices[i]])
        print('BRAND :', data['brand'].loc[df_indices[i]])
        print('Euclidean distance from given input image:', pdists[i])
        print('='*125)

avg_w2v_model(12566, 30)

ASIN : b0755pxb3g
BRAND : melissa mccarthy seven7
Euclidean distance from given input image: 0.0
ASIN : b0755lgtd9
BRAND : melissa mccarthy seven7
Euclidean distance from given input image: 0.46038264
ASIN : b01n7atp2x
BRAND : melissa mccarthy seven7
Euclidean distance from given input image: 0.62437814
ASIN : b010w7tqm8
BRAND : fringoo
Euclidean distance from given input image: 0.62624454
ASIN : b06xv1g5yr
BRAND : melissa mccarthy seven7
Euclidean distance from given input image: 0.6299955
ASIN : b018eb3q34
BRAND : juntong
Euclidean distance from given input image: 0.6407888
ASIN : b018e1zin0
BRAND : juntong
Euclidean distance from given input image: 0.6546518
ASIN : b018eb3nrs
BRAND : juntong
Euclidean distance from given input image: 0.6570116
ASIN : b018eb3upi
BRAND : juntong
Euclidean distance from given input image: 0.6615435
ASIN : b018eb3vra
BRAND : juntong
Euclidean distance from given input image: 0.6615435
ASIN : b010w7tq9g
BRAND : fringoo
Euclidean distance from given input